In [ ]:
#PigLab5NoteBook

! pip install pandas
! pip install openpyxl
!pip install rdflib
import pandas as pd
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

In [ ]:
# Define the file paths
files = ['data/PigData/Lab5/Decide-Exopol.xlsx']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_excel(file, engine='openpyxl')
    dfs.append(df)

df = dfs[0]


In [ ]:
df = df.drop(['Procedencia provincia'], axis=1)
df = df.drop(['Categoría animal'], axis=1)
df = df.drop(['Pool o individual'], axis=1)

In [ ]:
df

In [ ]:
#Rename Colums

df = df.rename(columns={
    'ID caso': 'SampleNumber',
    'Provincia': 'Province',
    'Fecha': 'Date',
    'PCR': 'Pathogen',
    'Resultado': 'Result',
    'Tipo de muestra': 'SampleType',
    'Edad': 'Age', #Age of animal
})

# Set DiagnosticTest to "PCR" Country to "Spain", Breed to "Meat", and LabReference to "1".
df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "5"






In [ ]:
#print(df['Age'].unique())

In [ ]:
#Annotation Mapping

# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "Positivo": "1",
    "positivo": "1",
    "Negativo": "0",
    "Neg.": "0",
    "Neg": "0",
    "neg.": "0",
    "negativo": "0",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")

PathogenMapping = {

    "Lawsonia intracellularis": "Lawsonia intracellularis",
    "L. intracellularis": "Lawsonia intracellularis",
    "L.intracelullaris": "Lawsonia intracellularis",
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Myc. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza A" : "Swine influenza virus",
  
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

# SampleTypeMapping for the additional sample types
SampleTypeMapping = {
    'Pulmón': 'Tissue',
    'Fluido oral': 'Fluid',
    'Hisopo': 'Swab',
    'Raspado': 'Tissue',
    'Lavado': 'other',
    'Cordón umbilical': 'Mixed',
    'Digestivo': 'Tissue',
    'Suero': 'Fluid',
    'Coriolis': 'other',
    'Muestra ambiental': 'other',
    'Heces': 'Feces',
    'Linfonodo': 'Tissue',
    'Lengua': 'Tissue',
    'Pulmón, Corazón': 'Mixed',
    'Pulmón, Bazo': 'Mixed',
    'Órganos feto': 'Tissue',
    'Pulmón, Linfonodo': 'Mixed',
    'Hisopo, Líquido pericárdico': 'Mixed',
    'Pulmón, Líquido pericárdico': 'Mixed',
    'Hígado': 'Tissue',
    'Pulmón, Hisopo': 'Mixed',
    'Pulmón, Corazón, Líquido pericárdico': 'Mixed',
    'Calzas': 'other',
    'Heces, Digestivo': 'Feces',
    'Intestino Delgado': 'Tissue',
    'Válvula Ileocecal': 'Tissue',
    'Pulmón, Ganglio nervioso': 'Mixed',
    'Toallita': 'Swab',
    'Articulación': 'Tissue',
    'Pulmón, Hígado': 'Mixed',
    'Pulmón, Tráquea, Tonsila': 'Mixed',
    'Pulmón, Tráquea': 'Mixed',
    'FTA': 'other'  # Example entry that might not fit into other categories
}

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Province mapping
ProvinceMapping = {
    'Castellón': 'Castellon',
    'Huesca': 'Huesca',
    'Zaragoza': 'Zaragoza',
    'Barcelona': 'Barcelona',
    'Albacete': 'Albacete',
    'Badajoz': 'Badajoz',
    'Sevilla': 'Seville',
    'Cáceres': 'Caceres',
    'Teruel': 'Teruel',
    'Murcia': 'Murcia',
    'Salamanca': 'Salamanca',
    'Girona': 'Girona',
    'Navarra': 'Navarre',
    'Toledo': 'Toledo',
    'Lleida': 'Lleida',
    'Zamora': 'Zamora',
    'Ávila': 'Avila',
    'Segovia': 'Segovia',
    'Valencia': 'Valencia',
    'Córdoba': 'Cordoba',
    'Lugo': 'Lugo',
    'Huelva': 'Huelva',
    'Málaga': 'Malaga',
    'Balears, Illes': 'Balears',
    'Gipuzkoa': 'Gipuzkoa',
    'Granada': 'Granada',
    'León': 'Leon',
    'Valladolid': 'Valladolid',
    'Almería': 'Almeria',
    'Jaén': 'Jaen',
    'Tarragona': 'Tarragona',
    'Soria': 'Soria',
    'Pontevedra': 'Pontevedra',
    'Madrid': 'Madrid',
    'Palmas, Las': 'Palmas',
    'ND': 'Missing',  # Handle special case for 'ND'
    'Alicante': 'Alicante',
    'Cuenca': 'Cuenca',
    'Ciudad Real': 'CiudadReal',
    'Santa Cruz de Tenerife': 'Santa Cruz de Tenerife',
    'Rioja, La': 'LaRioja',
    'Coruña, A': 'ACoruna',
    'Ourense': 'Ourense',
    'Cádiz': 'Cadiz',
    'Palencia': 'Palencia',
    'Burgos': 'Burgos',
    'Álava': 'Alava',
    'A Coruña': 'ACoruna',
    'Asturias': 'Asturias',
    'Bizkaia': 'Bizkaia',
    'Guadalajara': 'Guadalajara',
}


df["Province"] = df["Province"].map(ProvinceMapping).fillna("Missing")


In [ ]:
# PigAge mapping
PigStageMapping = {

    'Destetados': 'Weaning',
    'Cebo': 'Finishing',
    'ND': 'Missing',
    'Cebo ': 'Finishing',
    'Reposición (6 meses)': 'Sow',
    'Reposición ': 'Sow',
    'Transición 6 semanas': 'Finishing',
    '6-7 semanas': 'Nursery',
    'Cebo 9 semanas': 'Finishing',
    'Adultos ': 'Sow',
    'Cebo 40 a 150 días de vida. ': 'Finishing',
    'Adultos': 'Sow',
    'Transición (6 semanas)': 'Nursery',
    'Cebo (final)': 'Finishing',
    'Reposición (Gilt)': 'Gilt',
    'Reposición 16 y 20 semanas de vida': 'Sow',
    'Transición ': 'Finishing',
    'Destetados ': 'Weaning',
    'Lact.>15 ': 'Suckling',
    '4 meses. ': 'Finishing',
    'Ibérico Cebo (5-6 meses)': 'Finishing',
    '2-3 meses Ibérico': 'Finishing',
    'Cebo 14 semanas': 'Finishing',
    'Lechón': 'Suckling',
    'Adulto': 'Sow',
    '(Ibérico 2 meses)': 'Finishing',
    'Transición/Adultos ': 'Finishing',
    'Lact. ': 'Lactation',
    'Cebo (  3  meses) ': 'Finishing',
    'Transición (2 Meses)': 'Nursery',
    'Transición (7 Semanas)': 'Finishing',
    '6 Semanas': 'Finishing',
    'Transición (2-3 meses)': 'Nursery',
    'Lact. (2 días)': 'Suckling',
    'Transición (5 Semanas)': 'Finishing',
    'Cebo (4 meses) ': 'Finishing',
    'Transición (9-10 semanas)': 'Nursery',
    'Transición (6-7 semanas)': 'Nursery',
    'Desconocido': 'Other',
    'Transición 7 semanas': 'Finishing',
    'Cebo ( 3 meses) ': 'Finishing',
    'Cebo (100kg)': 'Finishing',
    'Lact. (20 días)': 'Suckling',
    'Cebo ( 2,5  meses) ': 'Finishing',
    'Cebo (futura reproductora)': 'Finishing',
    'Transición 25 días': 'Nursery',
    'Cebo 100Kg': 'Finishing',
    'Transición 8 semanas': 'Nursery',
    'Transición': 'Finishing',
    'Cebo (70Kg)': 'Finishing',
    'Lact. (<15 días) ': 'Suckling',
    'Cebo ( 1,5 meses)': 'Finishing',
    'Cebo ( 30-150kg.) ': 'Finishing',
    'Cebo ( 13 semanas) ': 'Nursery',
    'Entrada a Cebo ': 'Finishing',
    'Cebo ( meses)': 'Finishing',
    'Transición (30Kg)': 'Nursery',
    'Cebo (  4  meses) ': 'Finishing',
    'Cebo 45-50 kg.': 'Finishing',
    'Cuarentena': 'Other',
    'Cebo  ': 'Finishing',
    'Transición (8 semanas)': 'Nursery',
    'Cebo (30 kg) ': 'Finishing',
    'Transición (6 Semanas)': 'Nursery',
    'Cebo (8 semanas) ': 'Finishing',
    'Cebo (5 meses) ': 'Finishing',
    'Transición (4-5-5-7 semanas)': 'Nursery',
    'Transición (3-5-6-7 semanas) ': 'Nursery',
    'Transición (3-5-7 semanas)': 'Nursery',
    'Lact. Transición ': 'Suckling',
    'Cebo (3 meses)': 'Finishing',
    'Cebo (16 semanas)': 'Finishing',
    'Cebo (50Kg) ': 'Finishing',
    'Cebo (18 kg a entrada) ': 'Finishing',
    'Transición (5 semanas de vida)': 'Nursery',
    'Transición (9 semanas de vida)': 'Nursery',
    'Transición (12 semanas de vida)': 'Nursery',
    'Cebo (entrada a cebo)': 'Finishing',
    '3-5-7-9 semanas': 'Nursery',
    'Cebo (    meses) ': 'Finishing',
    'Transición (25-30 días)': 'Nursery',
    'Lact. Transición 2-5 semanas': 'Suckling',
    'Lechón ': 'Suckling',
    'Cebo ( 6 meses)': 'Finishing',
    'Lact. Transición (40 días)': 'Suckling',
    'Transición (7 semanas)': 'Finishing',
    'Primerizas ': 'Gilt',
    'Cebo (110 Kg)': 'Finishing',
    'Cebo (3 meses) ': 'Finishing',
    'Recría': 'Other',
    'Reposición': 'Sow',
    #Add new
    'Primerizas,Adultos': 'Sow',
    'Lact. (>15 días)': 'Suckling',
    'Lact.>15 (4 sem)': 'Suckling',
    'Cebo (13-14 semanas)': 'Finishing',
    'Cebo (2  meses)': 'Finishing',
    'Cebo (    meses) Transición': 'Finishing',
    'Cebo (    meses) Transición ': 'Finishing',
    'Cebo (12 semanas)': 'Finishing',
    'Cebo (5, 9 y 17 semanas)': 'Finishing',
    'Cebo (5, 9 y 17 semanas)': 'Finishing',
    'Cebo (90 días)': 'Finishing',
    'Cebo (    meses) Transición ': 'Finishing',
    'Cebo (    meses) Transición ': 'Finishing',
    'Cebo (22 semanas, 100 kg)': 'Finishing',
    'Cebo (22 semanas, 100 kg )': 'Finishing',
    'Cebo (17 semanas)': 'Finishing',
    'Transición (40 días)': 'Nursery',
    'Lact. (3 semanas)' : 'Suckling',
    'Cebo (90-100Kg)': 'Finishing',
    'Cebo (120 Kg) ': 'Finishing',
    'Cebo (120 Kg)': 'Finishing',
    'Cebo (90-100 Kg)': 'Finishing',
    'Cebo (40 kg) ': 'Finishing',
    'Cebo (60 kg) ': 'Finishing',
    'Cebo (100 kg)': 'Finishing',
    'Transición (6 Kg)': 'Nursery',
    'Cebo (12/13 semanas)': 'Finishing',
    'Cebo (entrada)': 'Finishing',
    'Cebo (30 kg) Ibérico': 'Finishing',
    'Cebo (50-60Kg). (16-17 Sem).': 'Finishing',
    'Transición (7 sem)': 'Finishing',
    'Transición/ Cuarentena': 'Other',
    'Cebo vida (90Kg)': 'Finishing',
    'Cebo (13 sem) ': 'Nursery',
    'Reposición (14-20 semanas)': 'Sow',
    'Cebo (15 sem) ': 'Finishing',
    'Cebo (2 meses)': 'Finishing',
    'Transición (5 Kg)': 'Nursery',
    'Adultos (1 año)': 'Sow',
    'Cebo 20 Kg.': 'Finishing',
    'Cebo (1 meses) ': 'Finishing',
    'Cebo (1,5 meses) Entrada a Cebo 7 Kg ': 'Finishing',
    'Transición (10-30 Kg).': 'Nursery',
    'Cebo (50 Kg)': 'Finishing',
    'Lact. (20  días) ': 'Suckling',
    '3-6-10 semanas': 'Nursery',
    'Cebo (2 meses) ': 'Finishing',
    'Cebo (6 semanas ) ': 'Finishing',
    'Cebo vida (50 kg)': 'Finishing',
    '11 semanas': 'Nursery',
    'Cebo (3,5 meses) ': 'Finishing',
    'Primerizas,Adultos': 'Sow',
    'Lact. ( 25 días)': 'Suckling',
    'Transición (6-8 Semanas)': 'Finishing',
    '3-6-9 semanas': 'Nursery',
    'Adultos (10 meses)': 'Sow',
    'Lact. (25 días) ': 'Suckling',
    'Lact. (25 días de vida) ': 'Suckling',
    'Cebo (entrada) ': 'Finishing',
    '5 semanas': 'Nursery',
    '3 semanas': 'Nursery',
    '9 semanas': 'Nursery',
    '4 semanas': 'Nursery',
    'Cebo (entrada cebo) ': 'Finishing',
    'Lact. (18 días)': 'Suckling',
    'Lact. Adulto': 'Sow',
    'Transición (Destetados)': 'Finishing',
    'Cebo ( 3  meses) ': 'Finishing',
    'Cebo (70 kg)': 'Finishing',
    'Cebo (90 kg) ': 'Finishing',
    'Cebo (1 año) ': 'Finishing',
    'Cebo (1 mes)': 'Finishing',
    'Transición (7-8 semanas)': 'Nursery',
    'Cebo (110 días en cebo)': 'Finishing',
    'Lechones': 'Suckling',
    'Destetados': 'Weaning',
    'Cebo': 'Finishing',
    'cebo': 'Finishing',
    'Cebo ': 'Finishing',
    'ND': 'Missing',
    'Reposición (6 meses)' : 'Sow',
    'Reposición': 'Sow',
    'Reposición ': 'Sow',
    'Transición 6 semanas': 'Finishing',
    'Cebo 9 semanas': 'Finishing',
    'Adultos': 'Sow',
    'Cebo 40 a 150 días de vida.': 'Finishing',
    'Transición (6 semanas)': 'Nursery',
    'Cebo (final)': 'Finishing',
    'Reposición (Gilt)': 'Gilt',
    'Reposición 16 y 20 semanas de vida': 'Sow',
    'Transición': 'Finishing',
    'Lact.>15': 'Sukling',
    '4 meses.': 'Finishing',
    'Ibérico Cebo (5-6 meses)': 'Finishing',
    '2-3 meses Ibérico': 'Finishing',
    'Lact.>15 (4 sem)': 'Suckling',
    'Cebo 14 semanas': 'Finishing',
    'Lechón': 'Suckling',
    '(Ibérico 2 meses)': 'Finishing',
    'Transición/Adultos': 'Finishing',
    'Lact.': 'Lactation',
    'Cebo (3 meses)': 'Finishing',
    'Transición (2 Meses)': 'Nursery',
    'Transición (7 Semanas)': 'Finishing',
    '6 Semanas': 'Finishing',
    'Transición (2-3 meses)': 'Nursey',
    'Lact. (2 días)': 'Sukling',
    'Transición (5 Semanas)': 'Finishing',
    'Cebo (4 meses)': 'Finishing',
    'Transición (9-10 semanas)': 'Nursery',
    'Desconocido': 'Other',
     'Cebo (3 meses)': 'Finishing',
    'Cebo (100kg)': 'Finishing',
    'Lact. (20 días)': 'Suckling',
    'Cebo (2,5 meses)': 'Finishing',
    'Cebo (futura reproductora)': 'Finishing',
    'Transición 25 días': 'Nursery',
    'Cebo 100Kg': 'Finishing',
    'Transición (8 semanas)': 'Nursery',
    'Cebo (30 kg)': 'Finishing',
    'Cebo (8 semanas)': 'Finishing',
    'Cebo (5 meses)': 'Finishing',
    'Transición (4-5-5-7 semanas)': 'Nursery',
    'Transición (3-5-6-7 semanas)': 'Nursery',
    'Transición (3-5-7 semanas)': 'Nursery',
    'Lact. Transición': 'Nursery',
    'Cebo (16 semanas)': 'Finishing',
    'Cebo (50Kg)': 'Finishing',
    'Cebo (18 kg a entrada)': 'Finishing',
    'Transición (5 semanas de vida)': 'Nursery',
    'Transición (9 semanas de vida)': 'Nursery',
    'Transición (12 semanas de vida)': 'Nursery',
    'Cebo (entrada a cebo)': 'Finishing ',
    'Cebo (meses)': 'Finishing',
    'Transición (30Kg)': 'Finishing',
    'Cebo (4 meses)': 'Finishing',
    'Cebo 45-50 kg.': 'Finishing',
    'Cuarentena': 'Other',
    'Cebo': 'Finishing',
    'Cebo (  4  meses)': 'Finishing',
    'Transición (6 Semanas)': 'Nursery',
    'Cebo (8 semanas)': 'Finishing',
    'Cebo (5 meses)': 'Finishing',
    'Cebo (70Kg)': 'Finishing',
    'Lact. (<15 días)': 'Sukling',
    'Cebo ( 1,5 meses)': 'Finishing',
    'Cebo (30-150kg.)': 'Finishing',
    'Cebo (13 semanas)': 'Finishing',
    'Entrada a Cebo': 'Finishing',
    'Cebo (meses)': 'Finishing',
    'Transición (30Kg)': 'Finishing',
    'Cebo (  4  meses)': 'Finishing',
    'Cebo 45-50 kg.': 'Finishing',
    'Cuarentena': 'other',
    'Lechón': 'Suckling',
    'Lact. Transición': 'Nursery',
    'Cebo': 'Finishing',
    'Reposición': 'Sow',
    'Reposición (6 meses)': 'Sow',
    'Transición (8 semanas)': 'Nursery',
    'Cebo (30 kg)': 'Finishing',
    'Transición (6 Semanas)': 'Nursery',
    'Cebo (8 semanas)': 'Finishing',
    'Cebo (5 meses)': 'Finishing',
    'Transición (4-5-5-7 semanas)': 'Nursery',
    'Transición (3-5-6-7 semanas)': 'Nursery',
    'Transición (3-5-7 semanas)': 'Nursery',
    'Lact. Transición': 'Nursery',
    'Cebo (3 meses)': 'Finishing',
    'Cebo (16 semanas)': 'Finishing',
    'Cebo (50Kg)': 'Finishing',
    'Cebo (18 kg a entrada)': 'Finishing',
    'Transición (5 semanas de vida)': 'Nursery',
    'Transición (9 semanas de vida)': 'Nursery',
    'Transición (12 semanas de vida)': 'Nursery',
    'Cebo (entrada a cebo)': 'Finishing',
    '3-5-7-9 semanas': 'Nursery',
    'Cebo (    meses)': 'Finishing',
    'Transición (25-30 días)': 'Nursery',
    'Lact. Transición 2-5 semanas': 'Nursery',
    'Cebo (6 meses)': 'Finishing',
    'Lact. Transición (40 días)': 'Nursery',
    'Transición (7 semanas)': 'Nursery',
    'Primerizas': 'Sow',
    'Cebo (110 Kg)': 'Finishing',
    'Cebo (3 meses)': 'Finishing',
    'Recría': 'Sow',
    'Transición (6 Kg)': 'Nursery',
    'Cebo (12/13 semanas)': 'Finishing',
    'Cebo (entrada)': 'Finishing',
    'Cebo (30 kg) Ibérico': 'Finishing',
    'Cebo (50-60Kg). (16-17 Sem).': 'Finishing',
    'Transición (7 sem)': 'Nursery',
    'Transición/ Cuarentena': 'Nursery',
    'Cebo vida (90Kg)': 'Finishing',
    'Cebo (13 sem)': 'Finishing',
    'Reposición (14-20 semanas)': 'Sow',
    'Cebo (15 sem)': 'Finishing',
    'Cebo (2 meses)': 'Finishing',
    'Transición (5 Kg)': 'Nursery',
    'Adultos (1 año)': 'Sow',
    'Cebo 20 Kg.': 'Finishing',
    'Cebo (1 meses)': 'Finishing',
    'Cebo (1,5 meses) Entrada a Cebo 7 Kg': 'Finishing',
    'Transición (10-30 Kg).': 'Nursery',
    'Cebo (50 Kg)': 'Finishing',
    'Lact. (20 días)': 'Suckling',
    '3-6-10 semanas': 'Nursery',
    'Cebo (2 meses)': 'Finishing',
    'Cebo (6 semanas)': 'Finishing',
    'Cebo vida (50 kg)': 'Finishing',
    '11 semanas': 'Nursery',
    'Cebo (3,5 meses)': 'Finishing',
    'Primerizas,Adultos': 'Sow',
    'Lact. (25 días)': 'Suckling',
    'Transición (6-8 Semanas)': 'Nursery',
    '3-6-9 semanas': 'Nursery',
    'Adultos (10 meses)': 'Sow',
    'Lact. (25 días)': 'Suckling',
    'Lact. (25 días de vida)': 'Suckling',
    'Cebo (entrada)': 'Finishing',
    '5 semanas': 'Nursery',
    '3 semanas': 'Nursery',
    '9 semanas': 'Nursery',
    '4 semanas': 'Nursery',
    'Cebo (entrada cebo)': 'Finishing',
    'Lact. (18 días)': 'Suckling',
    'Lact. Adulto': 'Sow',
    'Transición (Destetados)': 'Weaning',
    'Cebo ( 3  meses)': 'Finishing',
    'Cebo (70 kg)': 'Finishing',
    'Cebo (90 kg)': 'Finishing',
    'Cebo (1 año)': 'Finishing',
    'Cebo (1 mes)': 'Finishing',
    'Transición (7-8 semanas)': 'Nursery',
    'Cebo (110 días en cebo)': 'Finishing',
    'Transición (1 semana)': 'Nursery',
    'Cebo (14 semanas)': 'Finishing',
    'Lact. (>15 días)': 'Suckling',
    'Cebo (17 sem)': 'Finishing',
    'Cebo (13-14 semanas)': 'Finishing',
    'Cebo (17 semanas)': 'Finishing',
    'Transición (5 y 7 semanas)': 'Nursery',
    'Cebo (100 Kg)': 'Finishing',
    'Transición (40 días)': 'Nursery',
    'Desconocido': 'Other',
    'Lact. (3 semanas)': 'Suckling',
    'Cebo (5 meses)': 'Finishing',
    'Lact. (5 Kg)': 'Suckling',
    'Cebo (90-100Kg)': 'Finishing',
    'Transición 3 y 5 semanas': 'Nursery',
    'Cebo (40 kg)': 'Finishing',
    'Transición (2 meses)': 'Nursery',
    'Cebo (2  meses)': 'Finishing',
    'Cebo (    meses) Transición': 'Finishing',
    'Reposición (2º parto)': 'Sow',
    'Cebo (12 semanas)': 'Finishing',
    'Cebo (5, 9 y 17 semanas)': 'Finishing',
    'Cebo (90 días)': 'Finishing',
    'Cebo (4  meses)': 'Finishing',
    'Cebo (90-100 Kg)': 'Finishing',
    'Transición (5 y 8 semanas)': 'Nursery',
    'Transición (5 semanas)': 'Nursery',
    'Cebo (18 semanas)': 'Finishing',
    'Transición (5-7-9 semanas)': 'Nursery',
    'Cebo (100 kg)': 'Finishing',
    'Reposición (60-70 Kg)': 'Sow',
    'Cebo (  4  meses)': 'Finishing',
    'Transición (30 días)': 'Nursery',
    'Cebo (110 kg)': 'Finishing',
    'Transición (5 kg)': 'Nursery',
    'Transición (10 semanas)': 'Nursery',
    'Cebo (3-4 meses)': 'Finishing',
    'Lact. (    días)': 'Suckling',
    'Transición (9 sem)': 'Nursery',
    'Lact. (14  días)': 'Suckling',
    'Transición (3 semanas)': 'Nursery',
    'Cebo (12 meses)': 'Finishing',
    'Cebo (120 Kg)': 'Finishing',
    'Lact.': 'Suckling',
    'Transición (60 Kg)': 'Nursery',
    'Transición (25Kg)': 'Nursery',
    'Reproductoras': 'Sow',
    'Transición (40 días)': 'Nursery',
    'Cebo (22 semanas, 100 kg)': 'Finishing',
    'Cebo (7 meses, 80-90 kg)': 'Finishing',
    'Cebo (60 kg)': 'Finishing',
    'Transición (40-50 días)': 'Nursery',
    'Lact. Cebo, Transición': 'Nursery',
    'Adultos Transición (42 días)': 'Sow',
    'Adultos Transición': 'Sow',
    'Transición ( 7 semanas)': 'Nursery',
    'Transición (3-6-9-11 Semanas)': 'Nursery',
    'Transición (4 meses)': 'Nursery',
    'Lactantes': 'Suckling',
    'Gestantes': 'Sow',
    'Transición (9 semanas)': 'Nursery',
    'Transición (42 días)': 'Nursery',
    'Cebo (81 y 91 días)': 'Finishing',
    'Cebo (81 y 91 días )': 'Finishing',
    'cebo (81 y 91 días)': 'Finishing',
    'Cebo (180 kg)': 'Finishing',
    'Cebo (50 Kg)': 'Finishing',
    'Cebo (50 Kg)': 'Finishing',
    'Cebo (50 Kg) ': 'Finishing',
    'Cebo (50Kg)': 'Finishing',
    'Transición (3 y 6 semanas postdestete)': 'Nursery',
    'Cebo (100 Kg)': 'Finishing',
    'Cebo (100 Kg) ': 'Finishing',
    'Cebo (8-12 semanas)': 'Nursery',
    'Destetado': 'Weaning',
    'Destetado ': 'Weaning',
    'destetado ': 'Weaning',
    'destetado': 'Weaning',
    
    'Cebo (2,5 meses)': 'Finishing',
    'Cebo (2,5 meses) ': 'Finishing',
    'Cebo (vida)': 'Finishing',
    'Cebo (Híbridas)': 'Finishing',
    'Postweaning': 'Weaning',
    'Postweaning ': 'Weaning',
    'Transición (4 y 5 semanas de vida)': 'Nursery',
    'Cebo (10-30Kg.)': 'Finishing',
    'Cebo (10-30Kg.) ': 'Finishing',
    
    'Cebo ( 24 semanas de vida)': 'Nursery',
    'Transición (3 meses)': 'Nursery',
    'Cebo  (4  meses)': 'Finishing',
    'Cebo ( 4 meses)': 'Finishing',
    'Cebo (2 meses en cebo)': 'Finishing',
    'Cebo (16 semanas)': 'Finishing',
    'Cebo (40 Kg)': 'Finishing',
    'Cebo (2 meses en cebo)': 'Finishing',
    'Cebo (2 meses en cebo)': 'Finishing',
    'Cebo/Engorde (4 meses) ': 'Finishing',
    'Cebo/Engorde (4 meses)': 'Finishing',
    'Cebo/Engorde ( 1 mes)': 'Finishing',
    'Cebo/Engorde (4 días)': 'Finishing',
    'Cebo/Engorde ( 4 días)': 'Finishing',
    'Cebo/Engorde(4 días)': 'Finishing',
    'Cebo/Engorde (6 sem)': 'Finishing',
    'Cebo/Engorde (9 sem)': 'Finishing',

    'Cebo (4-5 meses)': 'Finishing',
    'Transición Cebo Reposición': 'Sow',
    'Cebo (4 meses)': 'Finishing',
    'Transición (5 semanas postdestete)': 'Nursery',
    'Transición (1 semanas postdestete)': 'Nursery',
    'Transición (6, 7 y 9 semanas)': 'Nursery',
    'Cebo (100 días)': 'Finishing',
    'Desconocido': 'Other',
    'Transición (4-9 semanas)': 'Nursery',
    'Cebo (115 días de vida)': 'Finishing',
    'Cebo (15 días en cebo)': 'Finishing',
    'Primerizas': 'Sow',
    'Transición (3-5-6-7 semanas)': 'Nursery',
    'Transición (3-4 semanas)': 'Nursery',
    'Lact. (3 semanas)': 'Suckling',
    'Transición (3-6-9 Semanas)': 'Nursery',
    'Lact. ( 21 días)': 'Suckling',
    'Cebo (1 mes y medio en cebo)': 'Finishing',
    'Destete': 'Weaning',
    'Cebo (20 sem)': 'Finishing',
    'Cebo (Entrada a cebo)': 'Finishing',
    'Cebo ( 3-4 meses)': 'Finishing',
    'Cebo ( 10 sem.)': 'Finishing',
    'Lact. (<15 días)  Adultos': 'Suckling',
    'Cebo (100 días )': 'Finishing',
    'Cebo (2,5 meses)': 'Finishing',
    'Transición (12 Kg)': 'Nursery',
    'Cebo Vida': 'Finishing',
    'Transición (2-3 sem postdestete)': 'Nursery',
    'Cebo (3 meses) Adultos': 'Finishing',
    'Cebo (6 meses)': 'Finishing',
    'Cebo (3,5 meses)': 'Finishing',
    'Cebo ( 3,5 meses)': 'Finishing',
    'Reposición (4-5 meses) Transición (8 semanas)': 'Sow',
    'Lact. (28 días)': 'Suckling',
    'Cebo (30 Kg)': 'Finishing',
    'Transición (15 días postdestete)': 'Nursery',
    'Adultos (Nulíparas)': 'Sow',
    'Transición (20 Y 28 días de vida)': 'Nursery',
    'Cebo (100 kg.)': 'Finishing',
    'Transición (15 Kg)': 'Nursery',
    'Lact (Final Lactación)': 'Suckling',
    'Lact. (12  días)': 'Suckling',
    'Transición (10-11 semanas de vida)': 'Nursery',
    'Transición (4, 5 y 6 semanas de vida)': 'Nursery',
    'Transición (4-5-6-7 semanas de vida)': 'Nursery',
    'Transición (6-7 semanas de vida)': 'Nursery',
    'Transición (5 y 6 semanas de vida)': 'Nursery',
    'Transición (5-6 y 7 semanas de vida)': 'Nursery',
    'Lact. (24 días)': 'Suckling',
    'Transición (6-9 semanas)': 'Nursery',
    'Cebo (8-20 semanas)': 'Finishing',
    'Lact. ( >21 días)': 'Suckling',
    'Transición (28 días de vida)': 'Nursery',
    'Cebo (3, 3,5, 4 y 5 meses)': 'Finishing',
    'Lact. ( <15   días)': 'Suckling',
    'Transición (7 semanas)': 'Nursery',
    'Transición Cebo': 'Finishing',
    'Transición (60 días)': 'Nursery',
    'Lact. (0 días) Transición': 'Suckling',
    'Transición (5-6 semanas de vida)': 'Nursery',
    'Transición (10 semanas de vida)': 'Nursery',
    'Nulíparas Multíparas': 'Sow',
    'Transición Reposición': 'Sow',
    'Cebo (3-6 meses)': 'Finishing',
    'Transición (3,5 meses; 20kg)': 'Nursery',
    'Cebo (    meses)': 'Finishing',
    'Transición (8-9 semanas)': 'Nursery',
    'Transición (6-7 sem de vida)': 'Nursery',
    'Lechones (15 kg)': 'Nursery',
    'Lechones (30 kg)': 'Nursery',
    'Cebo (50, 70 y 100 kg.)': 'Finishing',
    'Transición (1 sem. postdestete)': 'Nursery',
    'Cebo (22-23 semanas)': 'Finishing',
    'Lact. (35 días)': 'Suckling',
    'Adultos (Nulíparas)': 'Sow',
    'Desconocido': 'Other',
    'Transición (3, 6, 9 semanas de vida)': 'Nursery',
    'Ibérico Cebo (3 meses)': 'Finishing',
    ' Primerizas ': 'Sow',
    'Cebo (5 meses, 65kg)': 'Finishing',
    'Cebo (20 Kg)': 'Finishing',
    'Cebo (55 Kg)': 'Finishing',
    'Cebo (20-25 Kg)': 'Finishing',
    'Transición (3 días postdestete)': 'Nursery',
    'Ibérico (6 meses)': 'Finishing',
    "Cebo (5'5 meses)": 'Finishing',
    'Lact. (22-24 días)': 'Suckling',
    'Lact. ( 21-22 días)': 'Suckling',
    'Cebo vida ibérico': 'Finishing',
    'Lact. (25 días) Transición': 'Suckling',
    'Transición (8 semanas de vida)': 'Nursery',
    'Cebo (70 Kg)': 'Finishing',
    'Transición 4 semanas de vida': 'Nursery',
    'Cebo (12 semanas)': 'Finishing',
    'Cebo ( 6  meses)': 'Finishing',
    'Cebo ( 5 meses)': 'Finishing',
    'Cebo (150 días de vida)': 'Finishing',
    'Transición (21-24 días vida)': 'Nursery',
    'Adultos Lact.': 'Sow',
    'Cebo (Fase inicial cebo)': 'Finishing',
    'Transición (4 semanas)': 'Nursery',
    'Transición (80 días)': 'Nursery',
    'Transición (3-6 semanas postdestete)': 'Nursery',
    'Transición (5-7 semanas)': 'Nursery',
    'Cebo (1,5 meses)': 'Finishing',
    'Transición (70 días), Cebo (5 meses)': 'Nursery',
    'Lact. (23 días)': 'Suckling',
    'Cebo (   2 meses y medio)': 'Finishing',
    'Cebo (21 días)': 'Finishing',
    'Transición (1 semana postdestete)': 'Nursery',
    'Transición (36, 47 y 83 días de vida)': 'Nursery',
    '9 semanas de vida': 'Nursery',
    'Cebo (12 semanas de vida)': 'Finishing',
    'Cebo (4 meses de vida)': 'Finishing',
    'M150718': 'Other',
    'Adultos Transición ': 'Sow',
    'Reposición (Nulíparas)': 'Sow',
    'Transición (6 y 9 semanas)': 'Nursery',
    'Cebo 15 kg': 'Finishing',
    'Cebo (11 semanas de vida)': 'Finishing',
    'Transición (1, 2 y 3 semans)': 'Nursery',
    'Transición (4 semanas de vida)': 'Nursery',
    'Lact. (21 días)': 'Suckling',
    'Transición (7 semanas de vida)': 'Nursery',
    'Cebo (1 meses)': 'Finishing',
    'Cebo (45 Kg)': 'Finishing',
    'Adultos (Reproductoras)': 'Sow',
    'Cebo (15 semanas)': 'Finishing',
    'Cebo (3 meses en Cebadero)': 'Finishing',
    'Cebo ( 10 semanas)': 'Finishing',
    'Transición (7 Días Destetados)': 'Nursery',
    'Lact. (4 y 7 semanas)': 'Suckling',
    'Transición Adultos ': 'Sow',
    'Cebo (3  meses)': 'Finishing',
    'Verracos (>12 meses)': 'Boar',
    'Cebo (  3  meses)': 'Finishing',
    'Transición (6, 8 y 9 semanas)': 'Nursery',
    'Transición (12 semanas)': 'Nursery',
    'Reposición (5 meses)': 'Sow',
    'Cebo 2 meses': 'Finishing',
    'Cebo (120 días)': 'Finishing',
    'Destete Transición (4 semanas)': 'Nursery',
    'Transición (2-3 semanas)': 'Nursery',
    'Transición (4, 5, 6 y 8 semanas)': 'Nursery',
    'Adultos (Multípara)': 'Sow',
    'Transición (3-9 Semanas)': 'Nursery',
    'Transición 30 días': 'Nursery',
    'Transición (3, 6 y 7 semanas)': 'Nursery',
    'Lactantes (  14-21  días)': 'Suckling',
    'Transición 5 semanas': 'Nursery',
    '7 semanas': 'Nursery',
    'Cebo ( 2,5   meses)': 'Finishing',
    'Cebo (10 sem y 4 meses)': 'Finishing',
    'Transición (60 días de vida)': 'Nursery',
    'Transición (70 días)': 'Nursery',
    'Adultos >12 Meses': 'Sow',
    'Cebo (70 días)': 'Finishing',
    'Transición (6 y 8 semanas)': 'Nursery',
    'Destetado (3 semanas)': 'Weaning',
    'Destetado (40 días de vida)': 'Weaning',
    'Cebo (3 meses vida)': 'Finishing',
    'Cebo (18 kg.)': 'Finishing',
    'Cebo (inicio)': 'Finishing',
    'Cebo (inicio, 15 kg)': 'Finishing',
    'Cebo ( 11 Semanas) 25-30kg.': 'Finishing',
    'Destetado 8 Semanas Vida': 'Weaning',
    'Transición (50 kg)': 'Nursery',
    'Cebo ( 65 días)': 'Finishing',
    'Adultos verraco >12m': 'Boar',
    'Cebo (110 Kg)': 'Finishing',
    'Verraco >12 meses': 'Boar',
    'Transición (3-6-8/9 Semanas de vida)': 'Nursery',
    'Transición (8 Semanas de vida)': 'Nursery',
    'Adultos >12 meses': 'Sow',
    'Cebo (18 kg PV)': 'Finishing',
    '4-5-6-7-8-9 Semanas': 'Nursery',
    'Cebo (4,5  meses)': 'Finishing',
    'Cebo ( 2 semanas)': 'Finishing',
    'Transición (3 semanas de vida)': 'Nursery',
    'Transición (2-3, 4 y 5 semanas de vida)': 'Nursery',
    'Cebo (17 semanas)': 'Finishing',
    'Cebo (50-60 Kg)': 'Finishing',
    'Transición (3, 5 y 8 semanas)': 'Nursery',
    'Cebo (60 días)': 'Finishing',
    '4 semanas de vida': 'Nursery',
    'Transición (17 Kg)': 'Nursery',
    'Transición 7 y 8 semanas ': 'Nursery',
    'Transición 8 y 9 semanas de vida': 'Nursery',
    'Transición 6-9 semanas': 'Nursery',
    'Adultos (>12 meses)': 'Sow',
    'Cebo ( >18 Kg)': 'Finishing',
    'Adultos  Verraco >12 meses': 'Boar',
    'Cebo ( 20 Kg)': 'Finishing',
    'Lactantes (    días) Transición ': 'Suckling',
    'Cebo (13 y 24 semanas) ': 'Finishing',
    'Adultos (>12 meses) ': 'Sow',
    'Lactantes Transición ': 'Suckling',
    'Cebo (entrada hace 10 días) ': 'Finishing',
    'Transición 1 y 4 semanas de destetados': 'Nursery',
    'Cebo (21 días) ': 'Finishing',
    'Destetado (4-5 semanas postdestete)': 'Weaning',
    'Cebo ( 15 Kg.) ': 'Finishing',
    'Cebo (6,5 meses) ': 'Finishing',
    'Cebo (3 meses de vida) ': 'Finishing',
    'Lactantes (21 días)': 'Suckling',
    'Cebo (12 meses) ': 'Finishing',
    'Cebo (>12  meses) ': 'Finishing',
    'Transición (2 meses) ': 'Nursery',
    'Reposición 70 Kg.': 'Sow',
    'Lactantes (2-3 semanas)': 'Suckling',
    'Transición (5, 6, 7 y 8 semanas)': 'Nursery',
    'Cebo (60 Kg) ': 'Finishing',
    'Cebo (0,5 meses) ': 'Finishing',
    'Verracos > 12 meses': 'Boar',
    'Cebo (3 semanas en cebo) ': 'Finishing',
    'Lactantes (21 días) Transición ': 'Suckling',
    'Lactantes (17-20  días) ': 'Suckling',
    'Transición (5-6 y 7-8 semanas)': 'Nursery',
    'Adultos (5-6 meses)': 'Sow',
    'Transición Cebo ( 50 Kg.) ': 'Nursery',
    'Lactantes ': 'Suckling',
    'Cebo ( 60 Kg.) ': 'Finishing',
    'Cebo (45-50 Kg) ': 'Finishing',
    'Adultos >12 MESES': 'Sow',
    'Reposición Adultos Primerizas ': 'Sow',
    'Primerizas Reposición ': 'Sow',
    'Cebo (1.5 meses) ': 'Finishing',
    'Transición (3 y 4 semanas)': 'Nursery',
    'Transición  4 sem': 'Nursery',
    'Cebo (50-60 kg)': 'Finishing',
    'Cebo (>12 meses) ': 'Finishing',
    'Transición (40, 45 y 50 días, 8 semanas)': 'Nursery',
    'Transición (3, 4, 5, 6 semanas)': 'Nursery',
    'Destetado (3 semanas postdestete)': 'Weaning',
    'Cebo (30 Kg)': 'Finishing',
    'Destetado Cebo (3 meses) ': 'Weaning',
    'Cebo (  2  meses) ': 'Finishing',
    'Lactantes (19 días) ': 'Suckling',
    'Transición 4-5 Semanas': 'Nursery',
    'Transición 4-5 semanas': 'Nursery',
    'Cebo ( 4   meses) ': 'Finishing',
    'Cebo (8 meses) ': 'Finishing',
    'Cebo (10 meses)': 'Finishing',
    'Cebo ( 1   meses)': 'Finishing',
    'Destetado 1-2-3 semanas': 'Weaning',
    'Destetado (9 semanas)': 'Weaning',
    'Cebo (2 meses vida)': 'Finishing',
    'Cebo (6 meses) ': 'Finishing',
    'Cebo (1-2 meses) ': 'Finishing',
    'Cebo (3.5 meses) ': 'Finishing',
    'Lactantes (  10-12  días) ': 'Suckling',
    'Lactantes (> 15días)': 'Suckling',
    'Transición (5-6 sem vida)': 'Nursery',
    'Transición (2-8 sem vida)': 'Nursery',
    'Transición  y Adultos ': 'Sow',
    'Cebo (1 y 3 meses) ': 'Finishing',
    'Cebo (75-80 Kg) ': 'Finishing',
    'Transición (6 y 7 semanas)': 'Nursery',
    'Transición (3,6,9 semanas)': 'Nursery',
    'Cebo (50, 60, 70 y 80 días)': 'Finishing',
    'Cebo (80 días)': 'Finishing',
    'Destetado Cebo ': 'Weaning',
    'Transición (21, 42 y 63 días)': 'Nursery',
    'Cebo (20 Kg)': 'Finishing',
    'Transición (3, 5 y 6 sem) ': 'Nursery',
    '3-9 Semanas Transición ': 'Nursery',
    'Cebo (7 y 8 semanas) ': 'Finishing',
    'Lactantes (18 días) ': 'Suckling',
    'Cebo (24 Kg)': 'Finishing',
    'Nulíparas (7,5 meses)': 'Sow',
    'Cebo (25 y 80 Kg) ': 'Finishing',
    'Lactantes (4-7 y 18 días) ': 'Suckling',
    'Transición (5-6/ 7-8 semanas vida)': 'Nursery',
    'Transición Primerizas ': 'Nursery',
    'Transición (4-6 semanas de vida)': 'Nursery',
    'Destetado Reposición Adultos ': 'Sow',
    'Cebo (30, 60 y 100 Kg) ': 'Finishing',
    'Transición (3y 4 semanas) Cebo (7 y 8 semanas) ': 'Nursery',
    'Cebo (1,5 meses)': 'Finishing',
    'Cebo (80-100 Kg)': 'Finishing',
    'Primerizas Adultos ': 'Sow',
    'Reposición Destetados Lactantes': 'Sow',
    'Cebo (15-20 Kg.)': 'Finishing',
    'Transición (3 meses de vida)': 'Nursery',
    'Transición (7-8 semanas de vida)': 'Nursery',
    'Transición (6-7 Semanas de vida)': 'Nursery',
    'Transición (7 y 8 semanas)': 'Nursery',
    'Transición (4 y 6 semanas)': 'Nursery',
    'Transición (40, 50 y 75 días)': 'Nursery',
    'Cebo (12 Semanas)': 'Finishing',
    'Transición (5-9 semanas de vida)': 'Nursery',
    'Transición (7-8 Semanas de vida)': 'Nursery',
    'Transición (9 Semanas de vida)': 'Nursery',
    'Cebo (1 semana)': 'Finishing',
    'Cebo (10-15 kg.) ': 'Finishing',
    'Cebo ( 20-25 kg meses) ': 'Finishing',
    'Cebo ( 20-25 Kg) ': 'Finishing',
    'Cebo (15 días)': 'Finishing',
    'Lactantes (26 días) ': 'Suckling',
    'Cebo (25 Kg) ': 'Finishing',
    'Transición (45-60-80 días)': 'Nursery',
    'Transición (30-45-60 Días)': 'Nursery',
    'Transición (5,6 y 7 semanas)': 'Nursery',
    'Cebo (140 días) ': 'Finishing',
    'Cebo (90 Kg)': 'Finishing',
    'Destetado (21 días)': 'Weaning',
    'Verracos': 'Boar',
    'Destetado': 'Weaning',
    'Transición (1, 2, 5 y 9 semanas)': 'Nursery',
    'Cebo (25-30 Kg) ': 'Finishing',
    'Lactantes (20 días) ': 'Suckling',
    'Lactantes (5 y 10 días) ': 'Suckling',
    'Cebo (25-30 Kg.)': 'Finishing',
    'Lactantes (2 días) ': 'Suckling',
    'Cebo (15-16 kg.) ': 'Finishing',
    'Adultos > 12 Meses': 'Sow',
    'Cebo (40 Kg) ': 'Finishing',
    'Cebo (5 semanas) ': 'Finishing',
    'Cebo (15 Kg.) ': 'Finishing',
    'Destetado (10 -17 días postdestete)': 'Weaning',
    'Cebo (1 mes en cebo)': 'Finishing',
    'Cebo (3 meses en cebo)': 'Finishing',
    'Cebo (22 semanas) ': 'Finishing',
    'Transición (9-11 semanas)': 'Nursery',
    'Transición (23 Kg)': 'Nursery',
    'Destetado,Transición': 'Weaning',
    'Otra': 'Unknown',
    '3-5-7 Semanas': 'Nursery',
    'Transición (3, 5 y 7 semanas)': 'Nursery',
    'Transición (28-49 y 63 días)': 'Nursery',
    'Transiciónn (28-49 y 63 días)': 'Nursery',
    'Transición 10 Kg': 'Nursery',
    'Transición (6,9,12 semanas)': 'Nursery',
    'Destetado (20-30 días)': 'Weaning',
    'Transición (30 días de vida)': 'Nursery',
    'Cebo (12 semanas de vida)': 'Finishing',
    'Lactantes (10  días) ': 'Suckling',
    'Cebo (40 kg)': 'Finishing',
    'Nulíparas': 'Sow',
    ' Transición (5 y 7 semanas)': 'Nursery',
    'Lactantes (21-25 días)': 'Suckling',
    'Lactantes (2-35  días) ': 'Suckling',
    'Lactantes (2 días)': 'Suckling',
    'Lactantes (10 kg) ': 'Suckling',
    'Cebo ( 3 meses) Reproductoras': 'Finishing',
    'Cebo ( 2 meses) ': 'Finishing',
    'Cebo (15 Kg)': 'Finishing',
    'Lactantes Destetado ': 'Weaning',
    'Transición (3, 5, 7 y 9 semanas de vida)': 'Nursery',
    'Adultos Lactantes (NM) ': 'Sow',
    'Transición (10 Kg)': 'Nursery',
    'Cebo ( 80 Kg.)': 'Finishing',
    'Transición 8semanas': 'Nursery',
    'Transición (4, 6 y 8 semanas)': 'Nursery',
    'Lactantes (25 días)': 'Suckling',
    'Lactantes (25 días) ': 'Suckling',
    'Cebo (10 semanas) ': 'Finishing',
    'Transición (4-5 semanas)': 'Nursery',
    '8-9-10 semanas': 'Nursery',
    'Cebo (  15 Kg.)': 'Finishing',
    'Lactantes (21 días) ': 'Suckling',
    'Transición 3 semanas': 'Nursery',
    'Cebo (11 semanas) ': 'Finishing',
    'Transición (10-11 semanas)': 'Nursery',
    'Transición (30-50 días de vida)': 'Nursery',
    'Lactantes (10-20 días) ': 'Suckling',
    'Cebo (18-20 Kg) ': 'Finishing',
    'Cebo (14 semanas) ': 'Finishing',
    'Transición (4 y 8 semanas de vida)': 'Nursery',
    'Cebo (35 kg) ': 'Finishing',
    'Destetado (40-60 días vida)': 'Weaning',
    'Transición  (42 días vida) ': 'Nursery',
    'Transición  (65 días vida) ': 'Nursery',
    'Transición (65 días vida) ': 'Nursery',
    'Transición  (6 y 9 semanas) ': 'Nursery',
    'Transición (3 y 5 semanas vida)': 'Nursery',
    'Cebo (2´5 meses) ': 'Finishing',
    'Cebo (3 meses de vida)': 'Finishing',
    'Cebo (80 Kg) ': 'Finishing',
    'Destetado (5-10 semanas vida)': 'Weaning',
    'Cebo ( 2,5 meses) ': 'Finishing',
    'Cebo (25-30 Kg.) ': 'Finishing',
    'Cebo (16 semanas vida)': 'Finishing',
    'Lactantes (24  días)': 'Suckling',
    '4 Semanas': 'Nursery',
    'Transición Cebo ': 'Nursery',
    'Transición (35 días vida)': 'Nursery',
    'Lactantes (28 días)': 'Suckling',
    'Transición (5- 8 semanas vida)': 'Nursery',
    'Lactantes (21 días) Transición 5 Semanas': 'Suckling',
    'Cebo (3 y 5 meses) ': 'Finishing',
    'Cebo (8 semanas)': 'Finishing',
    'Cebo (6 semanas)': 'Finishing',
    'Cebo (9 semanas)': 'Finishing',
    'Lactantes ( días)': 'Suckling',
    'Cebo (2.5 meses)': 'Finishing',
    'Lactantes Destetado': 'Weaning',
    'Cebo 19 semanas': 'Finishing',
    'Cebo (120 días) ': 'Finishing',
    'Transición (45 días de vida)': 'Nursery',
    'Lactante (21 días) ': 'Suckling',
    'Cebo (  1,5  meses) ': 'Finishing',
    'Cebo Reposición ': 'Finishing',
    'Cebo (  2  meses)': 'Finishing',
    'Cebo ( 26 Semanas) ': 'Finishing',
    'Cebo ( 12-13 semanas) ': 'Finishing',
    'Transición (4, 5 y 6 semanas)': 'Nursery',
    'Cebo (  60 Kg)': 'Finishing',
    'Cebo (7 meses) ': 'Finishing',
    'Transición (2, 6 y 7 semanas)': 'Nursery',
    'Cebo ( 4 meses) ': 'Finishing',
    'Cebo ( 18 kg) ': 'Finishing',
    'Cebo (45 Kg) ': 'Finishing',
    'Transición (2, 3 y 5 semanas)': 'Nursery',
    'Cebo (  50 Kg)': 'Finishing',
    'Lactantes (21 días) Transición': 'Suckling',
    'Lactantes (    días) ': 'Suckling',
    'Cebo ( 2   meses) ': 'Finishing',
    'Cebo (,5 meses ) ': 'Finishing',
    'Cebo 5 meses': 'Finishing',
    'Cebo 4 meses': 'Finishing',
    'Adultos Lactantes (1 días) ': 'Sow',
    'Lactantes ( 7-15   días) ': 'Suckling',
    'Lactantes,Transición': 'Weaning',
    'Transición (8 semanas vida)': 'Nursery',
    'Lactantes (12 días)': 'Suckling',
    'Transición 9 semanas': 'Nursery',
    '8 semanas': 'Nursery',
    'Lactantes (15-21 días)': 'Suckling',
    'Cebo (40 Kg.)': 'Finishing',
    'Transición 5sdv': 'Unknown',
    'Destetado (2-3 semanas vida)': 'Weaning',
    'Lactantes (15 días)': 'Suckling',
    'Cebo (13 semanas)': 'Finishing',
    'Cebo (20Kg)': 'Finishing',
    'Cebo (6-10 semanas)': 'Finishing',
    'Lactantes': 'Suckling',
    'Cebo (90 Kg)': 'Finishing',
    'Destetado (60 días)': 'Weaning',
    'Lactantes (3 Semanas)': 'Suckling',
    'Reposición Lactantes (26 días)': 'Suckling',
    'Cebo (15-70Kg)': 'Finishing',
    'Lactantes (14 días)': 'Suckling',
    'Transición 7 semanas': 'Nursery',
    'Lactantes (días)': 'Suckling',
    'Lactantes (20 días)': 'Suckling',
    'Cebo (110 días)': 'Finishing',
    'Transición (2 semanas)': 'Nursery',
    'Destetado Adultos': 'Weaning',
    'Adultos (Verraco)': 'Boar',
    'Cebo (1 meses)': 'Finishing',
    'Transición 1 semana posdestete': 'Nursery',
    'Transición 3 y 4 sem posdestete': 'Nursery',
    'Destetado Transición': 'Weaning',
    'Cebo (16 Kg)': 'Finishing',
    'Destetado Transición (6 y 9 semanas)': 'Weaning',
    'Destetado, Transición (9-10 semanas)': 'Weaning',
    '7 y 8 semanas': 'Nursery',
    'Transición (4, 5, 6, 7 y 9 semanas)': 'Nursery',
    'Lactantes (21 días)': 'Suckling',
    'Cebo (1,5 meses)': 'Finishing',
    'Lactantes (26 días) Adultos': 'Suckling',
    'Transición 24-52 Días': 'Nursery',
    'Entrada Cebo': 'Finishing',
    'Cebo (18 Kg)': 'Finishing',
    'Transición (9 semanas)': 'Nursery',
    'Cebo (2,5 meses)': 'Finishing',
    'Lactantes (28 días)': 'Suckling',
    'Lactantes (1-10-20 días) Destetado (5 semanas de vida)': 'Weaning',
    'Cebo (2,5 meses)': 'Finishing',
    'Lactantes (28 días)': 'Suckling',
    'Lactantes (17 días)': 'Suckling',
    'Reposición 9 meses': 'Finishing',
    'Cebo (50 Kg.)': 'Finishing',
    'Adultos 90 Kg': 'Sow',
    'Transición (8-10 semanas)': 'Nursery',
    'Transición (6 semanas vida)': 'Nursery',
    'Transición (1, 2 y 5 semanas)': 'Nursery',
    'Transición (6-8 semanas)': 'Nursery',
    'Cebo (16-17 semanas)': 'Finishing',
    'Lactantes (14 días)': 'Suckling',
    '3-6-9 Semanas': 'Nursery',
    'Cebo (8-10 semanas)': 'Finishing',
    'Cebo (1 mes)': 'Finishing',
    'Lactantes (20 días)': 'Suckling',
    'Transición 28-63 Días': 'Nursery',
    'Transición (15-18 Kg) Cebo (7 meses)': 'Finishing',
    'Cebo (11 meses)': 'Finishing',
    'Cebo (5-6 meses)': 'Finishing',
    'Destetado (10 semanas de vida)': 'Weaning',
    'Destetado (9 semanas de vida)': 'Weaning',
    'Adultos Lactantes ( 15  días) Transición 3-9 semanas': 'Suckling',
    'Cebo (  1  meses)': 'Finishing',
    'Cebo (80 Kg.)': 'Finishing',
    'Lactantes (1 semanas)': 'Suckling',
    'Cebo (3 semanas)': 'Finishing',
    'Cebo (1  meses)': 'Finishing',
    'Destetado (33 días)': 'Weaning',
    'Transición 7w': 'Nursery',
    'Transición 6-7 semanas': 'Nursery',
    'Cebo (1 mes)': 'Finishing',
    'Cebo (3 Meses)': 'Finishing',
    'Cebo (  2,5  meses)': 'Finishing',
    'Transición 63d': 'Nursery',
    'Cebo (59 días)': 'Finishing',
    'Cebo (56 y 45 días)': 'Finishing',
    'Cebo (  9 semanas)': 'Finishing',
    'Lactantes ( 7  días)': 'Suckling',
    'Destetado (2-3 semanas)': 'Weaning',
    'Transición 4 semanas': 'Nursery',
    'Cebo (30 kg.)': 'Finishing',
    'Destetado (7 semanas de vida)': 'Weaning',
    'Cebo (  4 meses)': 'Finishing',
    'Lactantes (10 días)': 'Suckling',
    'Transición (1, 2, 3, 4 y 5 semanas)': 'Nursery',
    'Destetado  (30 y 70 días)': 'Weaning',
    'Transición Cebo (42 días)': 'Nursery',
    'Destetado (1 semana)': 'Weaning',
    'Cebo (18 semanas)': 'Finishing',
    'Destetado (28 días vida)': 'Weaning',
    'Transición (28 días vida)': 'Nursery',
    'Cebo (80 Kg)': 'Finishing',
    'Recría Transición': 'Nursery',
    'Lactantes (15 días)': 'Suckling',
    'Lactantes (28 días)': 'Suckling',
    'Transición (26 días en transición)': 'Nursery',
    'Destetado  (28 días)': 'Weaning',
    'Destetado (28 días)': 'Weaning',
    'Lactantes (21 días) Destetado': 'Weaning',
    'Lactantes (21  días)': 'Suckling',
    'Transición 3-5 Semanas': 'Nursery',
    'Transición (9-10 semanas)': 'Nursery',
    'Transición 6 semanas': 'Nursery',
    'Reproductoras': 'Sow',
    'Adultos (3 años)': 'Sow',
    'Destetado (4 sem vida)': 'Weaning',
    'Transición (7-8 sem vida)': 'Nursery',
    'Adultos (3-4 meses)': 'Sow',
    'Lactantes (24 días)': 'Suckling',
    'Destetado (30-60 días)': 'Weaning',
    'Cebo (30 Kg.)': 'Finishing',
    'Post Destetado': 'Nursery',
    'Reposición Transición': 'Nursery',
    'Transición 11 semanas': 'Nursery',
    'Destetado (4,5,6 y 7 sem)': 'Weaning',
    'Destetado (40 días vida)': 'Weaning',
    'Destetado (7-14 días)': 'Weaning',
    'Destetado (45 días)': 'Weaning',
    'Destetado (35 días)': 'Weaning',
    'Destetado (30 días)': 'Weaning',
    'Reproductoras (90 días gestación)': 'Sow',
    'Transición 10 semanas': 'Nursery',
    'Cebo ( 2 meses)': 'Finishing',
    'Cebo (  2,5  meses)': 'Finishing',
    'Cebo (15-16 Semanas)': 'Finishing',
    'Cebo (6-8-10 semanas)': 'Finishing',
    'Cebo (5-7-9 semanas)': 'Finishing',
    'Cebo (4-6-8-10 semanas)': 'Finishing',
    'Cebo ( 60 Días de vida)': 'Finishing',
    'Cebo (  3meses)': 'Finishing',
    'Destetado (20 días)': 'Weaning',
    'Lactantes ( 7 días)': 'Suckling',
    'Cebo (28 días)': 'Finishing',
    'Cebo (66 días)': 'Finishing',
    'Transición 28d': 'Nursery',
    'Cebo (18 semanas)': 'Finishing',
    'Cebo ( 1 meses)': 'Finishing',
     'Cebo (  9  meses)': 'Finishing',
    'Cebo (  1  meses)': 'Finishing',
    'Adultos Reproductoras': 'Sow',
    'Transición 7 Semanas': 'Nursery',
    'Cebo (6-7 semanas)': 'Finishing',
    '16 semanas': 'Finishing',
    'Lactantes (3 días)': 'Suckling',
    'Cebo (45 días)': 'Finishing',
    'Cebo ( 9-15 semanas)': 'Finishing',
    'Cebo ( 30-40 kg)': 'Finishing',
    'Cebo (30-40 Kg)': 'Finishing',
    'Cebo ( 17w)': 'Finishing',
    'Cebo (  40 kg.)': 'Finishing',
    'Cebo (35 Kg)': 'Finishing',
    'Lactantes (14 días) Transición': 'Suckling',
    'Cebo ( meses)': 'Finishing',
    'Cebo (20 semanas)': 'Finishing',
    'Cebo (25 semanas)': 'Finishing',
    'Cebo Adultos': 'Sow',
    'Transición 0-3-6 semanas': 'Nursery',
    'Lactantes ( 23 días)': 'Suckling',
    'Lactantes (20 Kg)': 'Suckling',
    'Cebo ( 30 kg.) ': 'Finishing',
    'Destetado (6 Kg)': 'Weaning',
    'Reproductoras (30 días gestación)': 'Sow',
    'Cebo ( 80 kg ) ': 'Finishing',
    'Cebo (8 meses - 100 Kg)': 'Finishing',
    'Cebo ( 50 Kg)': 'Finishing',
    'Lactantes (14 días)': 'Suckling',
    'Transición 4-5-9 semanas': 'Nursery',
    'Lactantes (22 días)': 'Suckling',
    'Adultos (1,5 años)': 'Sow',
    'Cebo (6 kg.)': 'Finishing',
    'Cebo ( 60-70 kg.) ': 'Finishing',
    'Cebo ( 4  meses)': 'Finishing',
    'Destetado (60-70 días vida)': 'Weaning',
    'Lactantes (1 días) Destetado': 'Weaning',
    'Destetado (6 y 9 semanas)': 'Weaning',
    'Reproductoras/es': 'Sow',
    'Cebo/Engorde ': 'Finishing',
    'Cebo/Engorde (11 sem)': 'Finishing',
    'Destetado/Transición ': 'Weaning',
    'Cebo/Engorde': 'Finishing',
    'Cebo/Engorde (2 meses)': 'Finishing',
    'Cebo/Engorde ( 7 meses   sem/mes/año)': 'Finishing',
    'Cebo/Engorde (  2 meses)': 'Finishing',
    'Destetado/Transición': 'Weaning',
    'Cebo/Engorde (35 días) ': 'Finishing',
    'Lactantes (  30  días)': 'Suckling',
    'Cebo/Engorde (1 mes)': 'Finishing',
    'Recría/Reposición': 'Nursery',
    'Cebo/Engorde (  3 meses) ': 'Finishing',
    'Cebo/Engorde (    sem/mes/año) ': 'Finishing',
    'Cebo/Engorde (sem/mes/año) ': 'Finishing',
    'Cebo/Engorde (2 meses, <60Kg)': 'Finishing',
    'Cebo/Engorde (2 semanas, <30 Kg)': 'Finishing',
    'Cebo/Engorde (3 meses) ': 'Finishing',
    'Cebo/Engorde ( 3 meses': 'Finishing',
    'Cebo/Engorde (2,5 meses)': 'Finishing',
    'Cebo/Engorde ( 16 sem) ': 'Finishing',
    'Destetado/Transición (3-6-9 sem)': 'Weaning',
    'Cebo (1,5 meses, <60 Kg)': 'Finishing',
    'Destetado/Transición (2 y 4 semanas) ': 'Weaning',
    'Cebo/Engorde ( 3 meses)': 'Finishing',
    'Cebo/Engorde (3 meses)': 'Finishing',
    'Cebo/Engorde (10 sem) ': 'Finishing',
    'Cebo/Engorde (3 mes) ': 'Finishing',
    'Cebo/Engorde ( 2 meses)': 'Finishing',
    'Destetado/Transición (6 y 9 semanas) Primerizas ': 'Weaning',
    'Cebo/Engorde (1 sem) ': 'Finishing',
    'Destetado/Transición (2 meses vida)': 'Weaning',
    'Lactantes ( 22  días)': 'Suckling',
    'Cebo/Engorde (2 sem/mes)': 'Finishing',
    'Cebo/Engorde (    sem/mes/año)': 'Finishing',
    'Cebo/Engorde 20 Kg.': 'Finishing',
    'Cebo/Engorde (2 meses)': 'Finishing',
    'Cebo/Engorde (15 días) ': 'Finishing',
    'Cebo/Engorde (6 meses)': 'Finishing',
    'Cebo/Engorde (1 mes) ': 'Finishing',
    'Cebo/Engorde ( 3 mes) ': 'Finishing',
    'Destetado/Transición (9 semanas)': 'Weaning',
    'Destetado/Transición (7 semanas)': 'Weaning',
    'Destetado/Transición (6 semanas)': 'Weaning',
    'Destetado/Transición 7 semanas': 'Weaning',
    'Cebo/Engorde (6 Meses)': 'Finishing',
    'Cebo/Engorde ( 4 meses ) ': 'Finishing',
    'Destetado/Transición Adultos': 'Sow',
    'Cebo/Engorde (4meses) ': 'Finishing',
    'Destetado/Transición 5 semanas': 'Weaning',
    'Destetado/Transición 5 sem': 'Weaning',
    'Cebo/Engorde ( 1 meses)': 'Finishing',
    'Cebo/Engorde ( 1 mes)': 'Finishing',
    'Cebo/Engorde ( 16 semanas)': 'Finishing',
    'Destetado/Transición 6 semanas': 'Weaning',
    'Cebo/Engorde ( sem/mes/año)': 'Finishing',
    'Cebo/Engorde ( 4 meses)': 'Finishing',
    'Recría/Reposición 5-6 meses': 'Nursery',
    'Cebo/Engorde (30 Kg.)': 'Finishing',
    'Recría/Reposición ': 'Nursery',
    'Cebo/Engorde ( 6 meses)': 'Finishing',
    'Cebo/Engorde (4 meses)': 'Finishing',
    'Cebo/Engorde ( 3 sem) ': 'Finishing',
    'Cebo/Engorde ( 10 semanas)': 'Finishing',
    'Transición 30-35 días': 'Nursery',
    'Cebo/Engorde (4 mes)': 'Finishing',
    'Destetado/Transición (3 semanas)': 'Weaning',
    'Adultos 18 meses': 'Sow',
    'Semanas 44, 45, 46 y 47': 'Finishing',
    'Cebo/Engorde (  8  sem)': 'Finishing',
    'Lactantes (  15  días)': 'Suckling',
    'Primalas': 'Sow',
    'Reproductoras/es, Adultos': 'Sow',
    'Cebo (10 semanas)': 'Finishing',
    'Cebo/Engorde (1 meses)': 'Finishing',
    'Cebo/Engorde (5 semanas)': 'Finishing',
    'Cebo/Engorde (3 semanas)': 'Finishing',
    'Cebo/Engorde (6 semanas)': 'Finishing',
    'Cebo/Engorde (5 meses)': 'Finishing',
    'Destetado/Transición (6 y 9 semanas)': 'Weaning',
    'Destetado/Transición Cebo 2, 5 meses': 'Weaning',
    'Destetado/Transición (7 y 14 semanas)': 'Weaning',
    'Reproductoras/es ': 'Sow',
    'Cebo/Engorde (6 sem)': 'Finishing',
    'Cebo/Engorde ( 9 sem)': 'Finishing',
    'Cebo/Engorde (50-60 kg)': 'Finishing',
    'Destetado/Transición (5 semanas)': 'Weaning',
    'Cebo/Engorde (14 sem)': 'Finishing',
    'Cebo/Engorde (4 días)': 'Finishing',
    '18 Kgs': 'Finishing',
    'Cebo/Engorde (41 días)': 'Finishing',
    'Destetado/Transición (42 días)': 'Weaning',
    'Cebo/Engorde (80 días)': 'Finishing',
    'Cebo/Engorde (21 semanas)': 'Finishing',
    'Destetado/Transición (35-40 días)': 'Weaning',
    'Cebo/Engorde (2 semanas)': 'Finishing',
    'Cebo/Engorde ( 1 mes)': 'Finishing',
    'Cebo/Engorde (4 meses)': 'Finishing',
  
}

# Map the 'Edad' column using the English translation mapping
df['PigStage'] = df['Age'].map(PigStageMapping).fillna("Finishing")
df = df.drop(['Age'], axis=1)



In [ ]:
df

In [ ]:
# Save to desired path
df.to_csv('output/PigLab5.csv', index=False)

In [ ]:
   barometer = df

In [ ]:
barometer

### Step 02: Create an RDF graph and namespaces.

In [ ]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

###  Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [ ]:
#For Pig Data frame RDF conversion
for index, row in barometer.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab5PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = row["Pathogen"]
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab5.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
turtle_data = g.serialize(format="turtle")
print(turtle_data)

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame: 

In [ ]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

### Step 05: Query the data from updated ontology 

In [ ]:

query = """
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO: <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT
  (strafter(str(?Sample), "#") AS ?SampleName)
  (strafter(str(?Pathogen), "#") AS ?PathogenName)
  (strafter(str(?Breed), "#") AS ?BreedName)
  (strafter(str(?SampleType), "#") AS ?SampleTypeName)
  (strafter(str(?DiagnosticTest), "#") AS ?DiagnosticTestName)
  (strafter(str(?PigProductionStages), "#") AS ?ProductionStage)
  (strafter(str(?Province), "#") AS ?ProvinceName)
  (strafter(str(?Country), "#") AS ?CountryName)
  (strafter(str(?SampleResult), "#") AS ?SampleResultName)

WHERE {
 {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  }
}
"""


# Execute the query and retrieve the results
results = g.query(query)

# Convert the results to a Pandas dataframe
data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "Sample", "Pathogen", "Breed", "SampleType", 
    "DiagnosticTest","PigProductionStage", "Province", "Country", "SampleResult"
])

# Display the dataframe
df
